In [14]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local[*]")\
        .appName('third task')\
        .getOrCreate()


## Подзадание А

### Загрузка данных

In [15]:
data = spark.read.json("/home/zanand/Study/PyFolder/bigdata/big_data/second_task/json/person/*.json")


In [16]:
data.show(5)


+----------+----------------+--------------------+-------------+------+-----------+--------+-----------+-----------+----------------+----------------+-------------------------+------------------------+--------------------+----------+--------------------+---------+---------------+----------+--------------------+-------------+----------------+------------------+--------+---------+--------+--------------------+--------+--------+--------------------+----------------+--------------+-----+--------------------+--------------------+-------------+-------------+------------------+-----------------+-------------+--------------------+-----------+-----------+-----------------+-------------+--------------+---------------+-----------------+---------+--------------------+--------------------+--------------------+----+----------------+---------------------+--------------------+-----------------+-------------------+------------------+--------------------+---------+-----------------+--------+-----------+

### First

In [17]:
count = data.rdd \
        .map(lambda lesson: (lesson["lecturer"], 1)) \
        .reduceByKey(lambda a, b: a + b) 

count.collect()


[('асс. ГУНЕНКОВ М.Ю.', 86),
 ('ст.пр. ШАРУН И.В.', 146),
 ('доц., к.н. ПРОХОРОВ П.В.', 72),
 ('ст.пр. МУНЬКО В.В.', 142)]

### Second

In [18]:
max = data.rdd \
        .map(lambda lesson: ((lesson["lecturer"], lesson["discipline"]), 1)) \
        .reduceByKey(lambda a, b: a + b) \
        .map(lambda data: (data[0][0], (data[0][1], data[1]))) \
        .reduceByKey(lambda a, b: a if a[1] > b[1] else b)
        

max.collect()


[('асс. ГУНЕНКОВ М.Ю.', ('Обработка естественного языка', 48)),
 ('доц., к.н. ПРОХОРОВ П.В.',
  ('Параллельное и распределенное программирование', 36)),
 ('ст.пр. ШАРУН И.В.',
  ('Объектно-ориентированный анализ и проектирование', 50)),
 ('ст.пр. МУНЬКО В.В.', ('Оптимизация и принятие решений', 61))]

### Third

In [19]:
max = data.rdd \
        .map(lambda lesson: ((lesson["lecturer"], lesson["auditorium"]), 1)) \
        .reduceByKey(lambda a, b: a + b) \
        .map(lambda data: (data[0][0], (data[0][1], data[1]))) \
        .reduceByKey(lambda a, b: a if a[1] > b[1] else b)
        

max.collect()


[('асс. ГУНЕНКОВ М.Ю.', ('Г-331', 21)),
 ('ст.пр. ШАРУН И.В.', ('8-201', 55)),
 ('доц., к.н. ПРОХОРОВ П.В.', ('Г-411', 21)),
 ('ст.пр. МУНЬКО В.В.', ('8-206', 40))]

### Fourth

In [20]:
max = data.rdd \
        .map(lambda lesson: ((lesson["lecturer"], lesson["kindOfWork"]), 1)) \
        .reduceByKey(lambda a, b: a + b)
        
max.collect()


[(('асс. ГУНЕНКОВ М.Ю.', 'Пересдача задолженностей'), 2),
 (('доц., к.н. ПРОХОРОВ П.В.', 'Лекция'), 36),
 (('ст.пр. МУНЬКО В.В.', 'Практические занятия'), 122),
 (('ст.пр. МУНЬКО В.В.', 'Лекция'), 6),
 (('ст.пр. ШАРУН И.В.', 'Лабораторные работы'), 88),
 (('асс. ГУНЕНКОВ М.Ю.', 'Практические занятия'), 32),
 (('доц., к.н. ПРОХОРОВ П.В.', 'Лабораторные работы'), 36),
 (('ст.пр. ШАРУН И.В.', 'Пересдача задолженностей'), 8),
 (('асс. ГУНЕНКОВ М.Ю.', 'Лекция'), 1),
 (('ст.пр. МУНЬКО В.В.', 'Лабораторные работы'), 2),
 (('ст.пр. МУНЬКО В.В.', 'Пересдача задолженностей'), 12),
 (('ст.пр. ШАРУН И.В.', 'Практические занятия'), 50),
 (('асс. ГУНЕНКОВ М.Ю.', 'Лабораторные работы'), 51)]

## Подзадание Б

Данные

In [21]:
from mrjob.job import MRJob
from mrjob.job import MRStep
from mrjob.protocol import JSONValueProtocol, TextValueProtocol
from urllib.request import urlopen
import json
from datetime import datetime, timedelta
import itertools

START_DATE = "2023.12.03"
END_DATE = "2023.12.30"
DATE_FORMAT = "%Y.%m.%d"

LECTURER_MATRIX = list()
AUDITORIUM_MATRIXES = dict()

AUDITORIUMS = {178: "8-201", 176: "8-204"}


Функции

In [22]:

def index_to_date(index):
    if index == 0:
        return "08:00"
    elif index == 1:
        return "09:40"
    elif index == 2:
        return "11:35"
    elif index == 3:
        return "13:15"
    elif index == 4:
        return "15:10"
    elif index == 5:
        return "16:50"
    elif index == 6:
        return "18:30"
    
def date_to_index(date):
    if date == "08:00":
        return 0
    elif date == "09:40":
        return 1
    elif date == "11:35":
        return 2
    elif date == "13:15":
        return 3
    elif date == "15:10":
        return 4
    elif date == "16:50":
        return 5
    elif date == "18:30":
        return 6

def matrix_from_lessons(lessons, start_date, end_date):
    start_date = datetime.strptime(start_date, DATE_FORMAT)
    end_date = datetime.strptime(end_date, DATE_FORMAT)

    days = (end_date - start_date).days + 1
    matrix =  [[True] * 7 for _ in range(0, days)]

    for lesson in lessons:
        time_index = date_to_index(lesson["beginLesson"])
        if time_index is None:
            continue

        date = datetime.strptime(lesson["date"], DATE_FORMAT)
        day_index = (date - start_date).days


        matrix[day_index][time_index] = False


    return matrix

def lessons_from_url(url):
 
    resp = urlopen(url)
    data = resp.read()
    return json.loads(data.decode("utf-8"))

def get_lecturer_matrix(id, start_date = START_DATE, end_date = END_DATE):
    url = f"https://rasp.omgtu.ru/api/schedule/person/{id}?start={start_date}&finish={end_date}&lng=1"
 
    lessons = lessons_from_url(url)
    matrix = matrix_from_lessons(lessons, start_date, end_date)
    return matrix

def get_auditorium_matrix(id, start_date = START_DATE, end_date = END_DATE):
    url = f"https://rasp.omgtu.ru/api/schedule/auditorium/{id}?start={start_date}&finish={end_date}&lng=1"

    lessons = lessons_from_url(url)
    matrix = matrix_from_lessons(lessons, start_date, end_date)
    return matrix

def get_group_matrix(id, start_date = START_DATE, end_date = END_DATE):
    url = f"https://rasp.omgtu.ru/api/schedule/group/{id}?start={start_date}&finish={end_date}&lng=1"
 
    lessons = lessons_from_url(url)
    matrix = matrix_from_lessons(lessons, start_date, end_date)
    return matrix

def group_reducer(data):
        group_id = data[0][0]
        group_name = data[0][1]
        group_matrix = get_group_matrix(group_id)
        lecturer_matrix = LECTURER_MATRIX

        start_date = datetime.strptime(START_DATE, DATE_FORMAT)
        end_date = datetime.strptime(END_DATE, DATE_FORMAT)
        days = (end_date - start_date).days + 1

        for lesson in data[1]:
            for (day, time) in itertools.product(range(0, days), range(0, 7)):
                
                for (audit_id, audit_matrix) in AUDITORIUM_MATRIXES.items():
                
                    if group_matrix[day][time] and lecturer_matrix[day][time] and audit_matrix[day][time]:
                        
                        # количество пар должно быть меньше 4
                        if(sum(group_matrix[day]) > 4):
                            continue
                        
                        # True - есть пара
                        # False - нет пары
                        prev_time = True if time <= 0 else group_matrix[day][time - 1]
                        next_time = True if time >= 6 else group_matrix[day][time + 1]
                        
                        # у пар не должно быть окон, значит после нашей и до нашей пары должна быть хоть одна пара
                        if prev_time and next_time:
                             continue
                        
                        lecturer_matrix[day][time] = False
                        audit_matrix[day][time] = False
                        group_matrix[day][time] = False
                        
                        yield f"[{group_name}] date:{lesson['date']}-{lesson['beginLesson']} => [{AUDITORIUMS[audit_id]}] {(start_date + timedelta(days=day)).date()}:{index_to_date(time)}"
                        break
                else:
                    continue
                break
            
            else:
                # if nothing found
                yield f"group:{group_id}, lesson: {lesson['date']}-{lesson['beginLesson']} => NO OPTIONS FOUND :("
                continue


Загрузка

In [24]:
LECTURER_MATRIX = get_lecturer_matrix(782898)
AUDITORIUM_MATRIXES[178] = get_auditorium_matrix(178) # 8-201
AUDITORIUM_MATRIXES[176] = get_auditorium_matrix(176) # 8-204

data = spark.read.json("/home/zanand/Study/PyFolder/bigdata/big_data/second_task/b_subtask/input.json")

data.show(5)


+----------+----------------+--------------------+-------------+------+-----------+--------------+-----------+-----------+----------------+----------------+-------------------------+------------------------+--------------------+----------+--------------------+---------+---------------+----------+--------------------+-------------+----------------+------------------+--------+---------+-------+--------------------+--------+--------+--------------------+----------------+--------------+-----+--------------------+--------------------+-------------+-------------+-----------------+-----------------+-------------+--------------------+-----------+-----------+-----------------+-------------+--------------+---------------+-----------------+---------+--------------------+--------------------+--------------------+----+----------------+---------------------+--------------------+--------+-------------------+------------------+------+---------+-----------------+--------+-----------+-------------------

In [25]:
result = data.rdd \
    .map(lambda lesson: ((lesson["groupOid"], lesson["group"]), [lesson]) ) \
    .reduceByKey(lambda lesson1, lesson2: lesson1 + lesson2) \
    .flatMap(group_reducer)
    
result.collect()


['[МО-201] date:2023.11.06-16:50 => [8-201] 2023-12-06:11:35',
 '[МО-201] date:2023.11.16-08:00 => [8-204] 2023-12-06:09:40',
 '[МО-201] date:2023.11.16-09:40 => [8-201] 2023-12-06:08:00',
 '[МО-201] date:2023.11.20-16:50 => [8-201] 2023-12-07:18:30',
 '[ФИТ-221] date:2023.11.07-11:35 => [8-204] 2023-12-05:08:00',
 '[ФИТ-221] date:2023.11.21-11:35 => [8-201] 2023-12-07:09:40',
 '[ФИТ-222] date:2023.11.07-13:15 => [8-201] 2023-12-12:08:00',
 '[ФИТ-222] date:2023.11.21-13:15 => [8-201] 2023-12-19:08:00',
 '[ФИТ-222] date:2023.11.22-11:35 => [8-201] 2023-12-20:11:35',
 '[МО-221] date:2023.11.10-15:10 => [8-201] 2023-12-06:16:50',
 '[МО-221] date:2023.11.24-15:10 => [8-201] 2023-12-08:18:30']